# Objectif

Je cherche une manière pratique de lire les objets .docx et des les séparer en plusieurs sections


In [185]:
filename = "/Users/benjaminhabert/Documents/Quantmetry_Missions/NewsLetter_Technique/2017-03-27_Technical_Newsletter.docx"

In [186]:
import mammoth

In [206]:
import re
def find_date(s):
    try:
        pattern = '(\d{4}).(\d{2}).(\d{2})'
        return '-'.join(re.search(pattern, s).groups())
    except AttributeError:
        return None

def find_title(raw, balise, text):
    if balise.startswith('h') :
        return text
    if balise == "p":
        try:
            pattern = r"^<strong>(.+?)</strong>$"
            result = re.match(pattern, text)
            return result.group(1)
        except Exception:
            pass
    return None


def strip_images(html):
    """ clean_html, image_list = strip_images(newsitem['rawhtml']) """
    # removing images
    pattern = r'<img src=".+?" />'
    without_images, _ = re.subn(pattern, '', html)
    # removing empty paragraphs
    pattern = r'<p></p>'
    without_images, _ = re.subn(pattern, '', without_images)
    # getting list of image names
    pattern = r'<img src="(.+?)" />'
    image_list = re.findall(pattern, html)
    return without_images, image_list
    
def conclude_newsitem(newsitem):
    clean_html, image_list = strip_images(newsitem['rawhtml'])
    newsitem['rawhtml'] = clean_html
    newsitem['images'] = image_list
    return newsitem
    
def html_to_newsitems(html):
    pattern = r"(<(?P<balise>.+?)>(.+?)</(?P=balise)>)"
    result = re.findall(pattern, html)
    date = find_date(result[0][2])
    print(date)
    news = []
    newsitem = None
    for raw, balise, text in result[1:]:
        title = find_title(raw, balise, text)
        if title:
            if newsitem:
                
                news.append(conclude_newsitem(newsitem))
            newsitem = {
                'title': title,
                'date': date,
                'rawhtml': ""
            }
        elif newsitem:
            newsitem['rawhtml'] += raw
    if newsitem:
        news.append(conclude_newsitem(newsitem))
    return news
    
    
def convert_image(image):
    # with image.open() as image_bytes:
    #     encoded_src = base64.b64encode(image_bytes.read()).decode("ascii")
    # STEP 1: save image to disc
    
    # STEP 2: return image info (location in app)
    return {
        # "src": "data:{0};base64,{1}".format(image.content_type, encoded_src)
        "src": 'location/in/app'
    }

def get_structured_document(filename):
    with open(filename, "rb") as docx_file:
        result = mammoth.convert_to_html(docx_file, convert_image=mammoth.images.img_element(convert_image))
        html = result.value # The generated HTML
        messages = result.messages # Any messages, such as warnings during conversion
        return html_to_newsitems(html)
    
structured = get_structured_document(filename)

2017-03-27


In [207]:
print(json.dumps(structured, indent=3))

[
   {
      "date": "2017-03-27",
      "title": "Servir un mod\u00e8le de machine learning avec une API Web",
      "rawhtml": "<p>La mise en production d\u2019un mod\u00e8le entra\u00een\u00e9 reste une probl\u00e9matique r\u00e9currente chez nos clients. Une solution possible est de cr\u00e9er un service web\u00a0: un serveur peut \u00eatre requ\u00eat\u00e9 via une API REST. Une requ\u00eate contenant les features est envoy\u00e9e au server\u00a0; celui-ci retourne la pr\u00e9diction. C\u2019est ce que propose le framework <a href=\"https://github.com/manigoswami/lightningbolt\">Lightning-Bolt</a>  qui s\u2019appuie sur Bottle pour la partie serveur et scikit-learn pour la pr\u00e9diction. </p>",
      "images": []
   },
   {
      "date": "2017-03-27",
      "title": "Apprentissage s\u00e9quentiel\u00a0: comment ne pas oublier",
      "rawhtml": "<p>L\u2019oubli est un probl\u00e8me qui se pose lors de l\u2019apprentissage s\u00e9quentiel de plusieurs t\u00e2ches. Par exemple ima

In [196]:
structured[-1]

{'date': '2017-03-27',
 'rawhtml': '<p><a href="https://github.com/AdeelK93/collapsibleTree">Ce package R</a> permet d’afficher des arbres interactifs (<a href="https://adeelk93.shinyapps.io/collapsibletree/">démo</a>)\xa0: on peut cliquer sur les nœuds de l’arbre pour les fermer / ouvrir. Ce package s’appuie sur le <a href="https://bl.ocks.org/mbostock/4339083">collapsibleTree</a> de D3.js. </p><p><img src="location/in/app" /></p>',
 'title': 'Afficher des arbres interactifs en R'}

In [197]:
t ='<p><a href="https://github.com/AdeelK93/collapsibleTree">Ce package R</a> permet d’afficher des arbres interactifs (<a href="https://adeelk93.shinyapps.io/collapsibletree/">démo</a>)\xa0: on peut cliquer sur les nœuds de l’arbre pour les fermer / ouvrir. Ce package s’appuie sur le <a href="https://bl.ocks.org/mbostock/4339083">collapsibleTree</a> de D3.js. </p><p><img src="location/in/app" /></p>'

'<'

In [199]:
strip_images(t)

('<p><a href="https://github.com/AdeelK93/collapsibleTree">Ce package R</a> permet d’afficher des arbres interactifs (<a href="https://adeelk93.shinyapps.io/collapsibletree/">démo</a>)\xa0: on peut cliquer sur les nœuds de l’arbre pour les fermer / ouvrir. Ce package s’appuie sur le <a href="https://bl.ocks.org/mbostock/4339083">collapsibleTree</a> de D3.js. </p>',
 ['location/in/app'])